In [1]:
import time
import torch
import gensim
import numpy as np

from torch.utils.data import DataLoader
from torch import optim

from torchtext.data import utils

from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
from dataset import AbstractDataset
from util import load_model_and_opt, batch_predict, load_from_cnn
from snn_util import AbstractHybrid, SpikingBCELoss, forward_pass, train_model

In [6]:
DEVICE = (f'cuda:0' if torch.cuda.is_available() else 'cpu')
HOME = '/home/hice1/khom9/CSE 8803 BMI Final Project'
CNN_VERSION = 6
# Path to the saved word embeddings file
EMBED_KEYS_PATH = f'{HOME}/wordvectors/abstracts200_trained_normalized_{CNN_VERSION}.wordvectors'

In [7]:
print(f'Using device {DEVICE}')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

Using device cuda:0
Tesla V100-PCIE-16GB


In [8]:
tk = utils.get_tokenizer('spacy') # use spacey tokenizer
wv = gensim.models.KeyedVectors.load(EMBED_KEYS_PATH, mmap='r') # Get the saved word vectors matrix
null_word = '\0' # Will be used to pad all abstracts to the same length (669 words)
d = AbstractDataset(f'{HOME}/CleanedAVdata.csv', 'Abstract', 'IPCR Classifications', tk, wv.key_to_index,
                    null_word=null_word, min_len=30, verbose=True) # PyTorch dataset for abstracts & their labels

100%|██████████| 23250/23250 [00:22<00:00, 1029.67it/s]


In [10]:
batch_size = 48
beta = 1.0
lr = 1e-3
T = 45

save_path = f'{HOME}/models/hybrid.pth'
cnn_path = f'{HOME}/models/cnn_model-{CNN_VERSION}.pth'
cnn_act_path = f'{HOME}/models/cnn_model-{CNN_VERSION}-max-activations.pkl'

loader = DataLoader(d, batch_size=batch_size, shuffle=True)
model = AbstractHybrid(T, EMBED_KEYS_PATH, null_word=null_word, beta=beta).to(DEVICE)

# Create a positive weight, such that we punish the model heavily for guessing 0 all the time.
num_pos = d.labels.sum(axis=0, keepdim=True).to_dense()
pos_weight = (d.labels.shape[0] - num_pos) / num_pos
# loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight.squeeze().to(DEVICE))
# loss_fn = snn.functional.ce_rate_loss()
# loss_fn = nn.BCELoss(weight=pos_weight.to(DEVICE))
loss_fn = SpikingBCELoss(pos_weight=pos_weight.squeeze().to(DEVICE), is_logits=True)

TypeError: SpikingBCELoss.__init__() got an unexpected keyword argument 'is_logits'

In [ ]:
model = load_from_cnn(model, cnn_path, cnn_act_path)
optimizer = optim.Adam(model.parameters(), lr=lr)
model, optimizer = load_model_and_opt(model, optimizer, save_path)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)
# optimizer.param_groups[0]['lr'] = 5e-5

In [ ]:
epochs = 250
train_model(model, optimizer, loss_fn, T=T, epochs=epochs, batch_size=batch_size, save_freq=25, save_path=save_path, 
            scheduler=scheduler)

In [ ]:
pred = (batch_predict(model, d.abst_data, T))
pred_spk = (torch.sigmoid(pred.mean(dim=0)) > 0.5).type(torch.float)
true = d.labels.to_dense()

In [ ]:
start = time.time()
total_loss = []
loss_fn_cpu = loss_fn.to('cpu')
for i in range(len(d)):
    total_loss.append(loss_fn_cpu(pred_spk[i].unsqueeze(0), true[i]).item())
    
print(f'Total avg loss: {np.mean(total_loss)}')

In [ ]:
# Print a sample prediction and true label
i = 8721 #8721 #18722
txt, label = d[i]
# txt = txt.unsqueeze(0)
loss_fn = loss_fn.to(DEVICE)
label = label.unsqueeze(0)

print(loss_fn(forward_pass(model, T, txt.to(DEVICE)).detach(), label.to(DEVICE)).item())
print(torch.cat([forward_pass(model, T, txt.to(DEVICE)).mean(dim=0).detach(), label.to(DEVICE)]).T)


In [ ]:
print(precision_score(true, pred_spk, average=None))
print(f'Total precision: {precision_score(true, pred_spk, average="weighted")}')

In [ ]:
print(recall_score(true, pred_spk, average=None))
print(f'Total recall: {recall_score(true, pred_spk, average="weighted")}')

In [ ]:
print(f'Total F1 score: {f1_score(true, pred_spk, average="weighted")}')

# 